# Version 1

In [ ]:
"""
@ Author: Zachary Deng
@ Date: 2021/1/5
@ Brief: 使用 DQN 算法训练CartPole-v0
"""

import random
import gym
import numpy as np
from tensorflow.keras import models, layers

env = gym.make('CartPole-v0')

STATE_DIM, ACTION_DIM = 4, 2 # State 维度 4, Action 维度 2

model = models.Sequential([
    layers.Dense(64, input_dim=STATE_DIM, activation='relu'),
    layers.Dense(20, activation='relu'),
    layers.Dense(ACTION_DIM, activation='linear')
])

model.summary()

In [ ]:
def generate_data_one_episode():
    '''生成单次游戏的训练数据'''
    x, y, score = [], [], 0
    obs = env.reset()
    while True:
        action = env.action_space.sample()
        x.append(obs)
        y.append([1,0] if action == 0 else [0,1])
        obs, reward, done, info = env.step(action)
        score += reward
        if done:
            break
    return x, y, score

def generate_training_data(expected_score=100):
    '''生成N次游戏的训练数据，并进行筛选，选择 > 100 的数据作为训练集'''
    data_X, data_Y, scores = [], [], []
    for i in range(10000):
        x, y, score = generate_data_one_episode()
        if score > expected_score:
            data_X += x
            data_Y += y
            scores.append(score)
    print('dataset size: {}, max score: {}'.format(len(data_X), max(scores)))
    return np.array(data_X), np.array(data_Y)

In [ ]:
data_X, data_Y = generate_training_data()
model.compile(loss='mse', optimizer='adam')
model.fit(data_X, data_Y, epochs=5)
model.save('CartPole-v0-nn.h5')

In [ ]:
saved_model = models.load_model('CartPole-v0-nn.h5')  # 加载模型
env = gym.make("CartPole-v0")  # 加载游戏环境

for i in range(5):
    obs = env.reset()
    score = 0
    while True:
        time.sleep(0.01)
        env.render()   # 显示画面
        action = np.argmax(saved_model.predict(np.array([obs]))[0])  # 预测动作
        obs, reward, done, info = env.step(action)  # 执行这个动作
        score += reward     # 每回合的得分
        if done:       # 游戏结束
            print('using nn, score: ', score)  # 打印分数
            break
env.close()

# Version 2

In [ ]:
import gym
import numpy as np 
import random
from collections import deque
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Parameters
ENV_NAME = 'CartPole-v0'
EPISODE = 10000
STEP = 300
TEST = 10 # The number of experiment test every 100 episode

# Hyper Parameters for DQN
GAMMA = 0.9 # discount factor for target Q
INITIAL_EPSILON = 0.5 # starting value of epsilon
FINAL_EPSILON = 0.01 # final value of epsilon
REPLAY_SIZE = 10000 # experience replay buffer size
BATCH_SIZE = 32 # size of minibatch

class DQN():
    # DQN Agent
    def __init__(self, env): #初始化
        #init experience replay
        self.replay_buffer = deque()
        
        #init some parameters
        self.time_step = 0
        self.epsilon = INITIAL_EPSILON
        self.state_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.n
        
        self.create_Q_network()
        self.create_training_method()

        # Init session
        self.session = tf.InteractiveSession()
        self.session.run(tf.initialize_all_variables())
        
    def create_Q_network(self): #创建Q网络
        # network weights
        W1 = self.weight_variable([self.state_dim,20])
        b1 = self.bias_variable([20])
        W2 = self.weight_variable([20,self.action_dim])
        b2 = self.bias_variable([self.action_dim])
        
        # input layer
        self.state_input = tf.placeholder("float",[None,self.state_dim])
        
        # hidden layers
        h_layer = tf.nn.relu(tf.matmul(self.state_input,W1) + b1)
        
        # Q Value layer
        self.Q_value = tf.matmul(h_layer,W2) + b2
        
    def weight_variable(self,shape):
        initial = tf.truncated_normal(shape)
        return tf.Variable(initial)

    def bias_variable(self,shape):
        initial = tf.constant(0.01, shape = shape)
        return tf.Variable(initial)
    
    def create_training_method(self): #创建训练方法
        self.action_input = tf.placeholder("float",[None,self.action_dim]) # one hot presentation
        self.y_input = tf.placeholder("float",[None])
        Q_action = tf.reduce_sum(tf.multiply(self.Q_value,self.action_input),reduction_indices = 1)
        self.cost = tf.reduce_mean(tf.square(self.y_input - Q_action))
        self.optimizer = tf.train.AdamOptimizer(0.0001).minimize(self.cost)
    
    def perceive(self, state, action, reward, next_state, done): #感知存储信息
        # one hot key format
        one_hot_action = np.zeros(self.action_dim)
        one_hot_action[action] = 1
        self.replay_buffer.append((state,one_hot_action,reward,next_state,done))
        if len(self.replay_buffer) > REPLAY_SIZE:
            self.replay_buffer.popleft()

        if len(self.replay_buffer) > BATCH_SIZE:
            self.train_Q_network()
            
    def train_Q_network(self): #训练网络
        self.time_step += 1
        # Step 1: obtain random minibatch from replay memory
        minibatch = random.sample(self.replay_buffer,BATCH_SIZE)
        state_batch = [data[0] for data in minibatch]
        action_batch = [data[1] for data in minibatch]
        reward_batch = [data[2] for data in minibatch]
        next_state_batch = [data[3] for data in minibatch]
    
        # Step 2: calculate y
        y_batch = []
        Q_value_batch = self.Q_value.eval(feed_dict={self.state_input:next_state_batch})
        for i in range(0,BATCH_SIZE):
            done = minibatch[i][4]
            if done:
                y_batch.append(reward_batch[i])
            else :
                y_batch.append(reward_batch[i] + GAMMA * np.max(Q_value_batch[i]))

        self.optimizer.run(feed_dict={
            self.y_input:y_batch,
            self.action_input:action_batch,
            self.state_input:state_batch
            })
        
    def egreedy_action(self,state): #输出带随机的动作
        Q_value = self.Q_value.eval(feed_dict = {
            self.state_input:[state]
            })[0]
        if random.random() <= self.epsilon:
            return random.randint(0,self.action_dim - 1)
        else:
            return np.argmax(Q_value)

        self.epsilon -= (INITIAL_EPSILON - FINAL_EPSILON)/10000
    
    def action(self, state): #输出动作
        return np.argmax(self.Q_value.eval(feed_dict = {
            self.state_input:[state]
            })[0])
    
# main function
def main():
    # initialize OpenAI Gym env and DQN agent
    env = gym.make(ENV_NAME)
    agent = DQN(env)
    
    for episode in range(EPISODE):
        # initialize task
        state = env.reset()
        
        # Train
        for step in range(STEP):
            # e-greedy action for train
            action = agent.egreedy_action(state) #输出动作
            next_state, reward, done, info = env.step(action)
            
            # Define reward for agent
            reward_agent = -1 if done else 0.1
            
            agent.perceive(state,action,reward,next_state,done) #输入状态
            state = next_state
            if done:
                break
                
        # Test every 100 episodes
        if episode%100 == 0:
            total_reward = 0
            for i in range(TEST):
                state = env.reset()
                for j in range(STEP):
                    env.render()
                    action = agent.action(state) #direct action for test
                    state, reward, done, info = env.step(action)
                    total_reward += reward
                    if done:
                        break
            ave_reward = total_reward/TEST
            print('episode:',episode,'    Evaluation Average Reward:',ave_reward)
            if ave_reward>=200:
                break
            
if __name__ == '__main__':
    main()
    
